##Подключаем гугл диск и стягиваем актуальные данные с него в гугл коллаб:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Копирование датасета с гугл диска:

In [ ]:
!cp /content/drive/MyDrive/bc/data.csv .

##Скачивание и импортирование библиотек:

In [ ]:
!pip install navec

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re

import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

from navec import Navec

##Определяем максимальную длину комментария (кол-во слов) и размер вектора:

In [ ]:
max_comment_len = 12
vector_size = 300

##Скачиваем или стягиваем векторные представления слов от Яндекса

In [ ]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar -O navec_hudlit_v1_12B_500K_300d_100q.tar

In [ ]:
navec = Navec.load('/content/drive/MyDrive/bc/navec_hudlit_v1_12B_500K_300d_100q.tar')

## Загружаем данные в память и убираем пустые строки



In [ ]:
data = open('data.csv')

In [ ]:
dataset = pd.read_csv(data)

In [ ]:
dataset

In [ ]:
dataset = dataset.dropna()

In [ ]:
dataset

## Очищаем и представляем отзывы как вектора

In [ ]:
russian_stopwords = stopwords.words("russian")

In [ ]:
important_words = ['не', 'и', 'или', 'но']

In [ ]:
for word in important_words:
  if word in russian_stopwords:
    russian_stopwords.remove(word)

In [ ]:
def clear_text(text: str) -> str:
  cleared_text = re.sub(r'[^А-яЁё]+', ' ', text).lower()
  return ' '.join(cleared_text.split())

def tokenize(text: str) -> list:
  tokens = text.split()
  passed_tokens = []
  for token in tokens:
    if not(token in russian_stopwords):
      passed_tokens.append(token)
  return passed_tokens

In [ ]:
dataset['Clear_comment'] = dataset['Comment'].apply(lambda x: tokenize(clear_text(str(x))))

In [ ]:
def vectorize_tokens(tokens, navec, max_len):
    unk = navec['<unk>']
    text_embeddings = []
    for tocken in tokens:
        embedding = navec.get(tocken, unk)
        text_embeddings.append(embedding)
    l = len(text_embeddings)
    if l > max_len:
        text_embeddings = text_embeddings[:max_len]
    else:
        text_embeddings = [navec['<pad>']] * (max_len - l) + text_embeddings
    return text_embeddings

In [ ]:
dataset['Vectors'] = dataset['Clear_comment'].apply(lambda x: vectorize_tokens(x, navec, max_comment_len))

In [ ]:
dataset

##Выделяем данные для обучения и теста, сохраняем ответы к ним:

In [ ]:
train = dataset[1000:]

In [ ]:
y_train = train['isPositive'].astype('int')

In [ ]:
file = open('/content/drive/MyDrive/bc/y_train', 'wb')
np.save(file, y_train)
file.close()

In [ ]:
test = dataset[:1000]

In [ ]:
y_test = test['isPositive'].astype('int')

In [ ]:
file = open('/content/drive/MyDrive/bc/y_test', 'wb')
np.save(file, y_test)
file.close()

##Меняем размерность векторных представлений (по частям, чтобы не зависло), сохраняем их:

In [ ]:
x_alltrain = np.array(train['Vectors'].tolist())

In [ ]:
x_train1 = x_alltrain[:10000].reshape(len(x_alltrain[:10000]), max_comment_len * vector_size)

In [ ]:
x_train2 = x_alltrain[10000:20000].reshape(len(x_alltrain[10000:20000]), max_comment_len * vector_size)

In [ ]:
x_train3 = x_alltrain[20000:30000].reshape(len(x_alltrain[20000:30000]), max_comment_len * vector_size)

In [ ]:
x_train4 = x_alltrain[30000:].reshape(len(x_alltrain[30000:]), max_comment_len * vector_size)

In [ ]:
x_train = np.vstack([x_train1, x_train2])

In [ ]:
x_train = np.vstack([x_train, x_train3])

In [ ]:
x_train = np.vstack([x_train, x_train4])

In [ ]:
x_train.shape

In [ ]:
file = open('/content/drive/MyDrive/bc/x_train', 'wb')
np.save(file, x_train)
file.close()

In [ ]:
x_alltest = np.array(test['Vectors'].tolist())

In [ ]:
x_test = x_alltest.reshape(len(x_alltest), max_comment_len * vector_size)

In [ ]:

x_test.shape

(1000, 3600)

In [ ]:
file = open('/content/drive/MyDrive/bc/x_test', 'wb')
np.save(file, x_test)
file.close()